![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: [Saving your work in Prompt Lab as a notebook](/docs).

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Creating an access token from the IBM Cloud personal API key
* Defining a Python class for calling the WML Foundation Model inferencing API
* Using the class to generate output from a provided text input

# Setup

## Inferencing class
This cell defines a class that makes a REST API call to the watsonx Foundation Model
inferencing API that we will use to generate output from the provided input.
The class takes the access token created in the previous step, and uses it to
make a REST API call with input, model id and model parameters. The response
from the API call is returned as the cell output.

In [ ]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [ ]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

# Inferencing
This cell demonstrated how we can use the defined class as well as the
created access token to pair it with model id, parameters and input string
to obtain the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "google/flan-ul2"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 300,
    "min_new_tokens": 50,
    "repetition_penalty": 2
}

## Defining the project id
The API requires project id that provides the context for the call. We will obtain
the id from the project in which this notebook runs:

In [ ]:
import os

project_id = os.environ["PROJECT_ID"]

## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:

In [ ]:
prompt_input = """The following document is a description of Missouri University's informatics department.  Read the document and then write a short 1 paragraph summary.

Document:

A Message from the Director

Welcome to University of Missouri Institute for Data Science and Informatics (MUIDSI) web site!  MUIDSI is an interdisciplinary research and graduate education program offering a PhD in Informatics and an MS in Data Science & Analytics. Trainees have the option to choose bioinformatics, biomedical/health informatics, or geoinformatics emphasis areas for their PhD studies. The MS in Data Science and Analytics program offers several emphasis areas for both executive/on-line and on-campus programs. Managed and supported by the Vice Chancellor for Research Office and the Graduate School, the Institute has more than 60 core faculty members from 8 colleges/schools mentoring 120 graduate students. Among the current student body, 42 PhD students are sponsored by extramural funding, training grants, industry sponsorships, etc.  Incoming PhD students are encouraged to also pursue an MS degree in Data Science & Analytics as part of their PhD training.

Our Informatics PhD students come from a wide spectrum of academic backgrounds, such as physicians, nurses, computer scientists, data scientists, biologists, chemists, statisticians, engineers, social scientists, etc. In addition, the MD-PhD Informatics program was just launched in 2020 to recruit prospective medical students.

Our MS in Data Science and Analytics (DSA) program offers state-of-the-art Big Data Analytics training for working professionals and residential students for career advancement to secure data scientist positions in various sectors.  Our current emphasis areas include Geospatial Analytics, Biohealth Analytics, Data Journalism/Strategic Communication, High Performance Computing, and Human-Centered Data Design.  The online MS DSA program received the 2018 University Professional and Continuing Education Association (UPCEA) Central Region Outstanding Credit Program Award and ranked #7 on Fortune’s Best Online Master’s in Data Science Programs in 2022.

If you are a prospective student, please feel free to contact me and our leadership/administrative team for more information on the application process. We are always looking for top-notch students to join our fantastic faculty’s labs and centers.  If you are seeking tenure-track faculty positions with MUIDSI and partnering academic units, we have a strong track record mentoring junior faculty with NSF CAREER program and NIH K awards through our interdisciplinary research network. It is my honor to work with such talented faculty, students, alumni, and staff, as well as all the outstanding people associated with and supporting MUIDSI. No bias – informatics and data science rule!

Chi-Ren Shyu, Ph.D.

Director and Professor, Institute for Data Science and Informatics
Cutting-Edge Research

Established in 2008, MUIDSI has led the way in cutting edge informatics research.

We host 60 core faculty members from 24 departments and 9 colleges/schools that support our unique interdisciplinary doctoral degree program with emphasis/concentration areas in bioinformatics, health informatics, and geoinformatics.

As of Fall 2021, we will have 50 doctoral students in the program, with various backgrounds in biology, computer science, engineering, geography, medicine, management, and public health.  Our faculty and students conduct active informatics research with extramural funding from the NIH, NSF, AHRQ, USDA, CMS, US Department of Education, and other industry sponsors.

MUIDSI has one of the 16 NIH BD2K institutional pre-doctoral training sites in the country. Our graduates report a 100% placement rate, holding positions as ranked faculty and postdocs in academia, as well as researchers at National Laboratories and private sectors in business and industry. In addition to our PhD program, the Institute also hosts a newly approved Masters Degree in Data Science and Analytics.
 
Our Mission: Overview

Building on a tradition of outstanding informatics education and research at Missouri, the MU Institute for Data Science and Informatics is an interdisciplinary research and education program supported by 50 core faculty members from 24 departments and 9 colleges/schools, such as:

    College of Engineering
    School of Medicine
    Sinclair School of Nursing
    College of Arts and Sciences
    College of Agriculture
    Food & Natural Resources
    School of Health Professionals
    College of Education

The Institute, under the governance of the Graduate School, offers a doctoral degree program in the areas of bioinformatics, health informatics, and geoinformatics. Each concentration stresses the skill sets and research appropriate for the subfield within the broad area of informatics.
Our Mission: Components
Our mission has three components: education, research, and outreach.

To have a global impact in advancing computational research in biology and medicine, the stakeholders across the University of Missouri System believe it is critical to the mission of the university and the development of the community to offer a doctoral program in informatics, build an international-level research program, and provide service to scientific communities for informatics needs.
Education

    Providing an interdisciplinary training environment that leverages the unique strengths of the Columbia campus and other University of Missouri campuses.
    Providing students with the core competencies and research skills to be productive and independent investigators in academia, industry, and public research institutions.
    Meeting informatics workforce needs in research within the State of Missouri, the Midwest region, and at the national and international levels.

Research

    Building teams of faculty members and professionals to foster main informatics research thrusts.
    Developing signature informatics research programs to seamlessly align with the local and international research trends in life sciences, geospatial sciences, and medicine.
    Producing high impact informatics research in the forms of, extramural funding, publication, and international recognitions.

Outreach

    Partnering and sharing informatics resources with other campuses of the University of Missouri System, regional partners, and worldwide collaborators.
    Disseminating informatics tools, providing web servers for advanced informatics analyses/search, and making research results/data available for others.
    Supporting entrepreneurial activities to develop high tech industry in the region.

Summary:
"""

## Execution
Let us now use the defined variables to create an instance of the class
we defined previously and get the response from the selected foundation model:

In [ ]:
prompt = Prompt(access_token, project_id)

prompt.generate(prompt_input, model_id, parameters)

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing API to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>  